# P7 : Détectez les Bad Buzz grâce au Deep Learning

## Préparation jeu de données modèle simple

In [1]:
import os
import pandas as pd
df_dep = pd.read_csv('P7_03_fichiercsv_tweet_val_lab_mod.csv')
df_dep.head()

C:\Users\JK253\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\JK253\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\JK253\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


,label,tweet,date_du_poste,utilisateur
0,0,is upset that he can't update his Facebook by ...,Mon Apr 06 22:19:49 PDT 2009,scotthamilton
1,0,@Kenichan I dived many times for the ball. Man...,Mon Apr 06 22:19:53 PDT 2009,mattycus
2,0,my whole body feels itchy and like its on fire,Mon Apr 06 22:19:57 PDT 2009,ElleCTF
3,0,"@nationwideclass no, it's not behaving at all....",Mon Apr 06 22:19:57 PDT 2009,Karoli
4,0,@Kwesidei not the whole crew,Mon Apr 06 22:20:00 PDT 2009,joy_wolf


In [2]:
df_dep.label.value_counts()

1    800000
0    799999
Name: label, dtype: int64

In [3]:
# Extrait des données
n_sample = 5000
df_neg= df_dep[df_dep['label'] == 0].sample(n=n_sample)
df_pos = df_dep[df_dep['label'] == 1].sample(n=n_sample)

data = pd.concat([df_neg, df_pos], ignore_index=True)

In [4]:
# Extrait des données2
n_sample = 500
df_neg= data[data['label'] == 0].sample(n=n_sample)
df_pos = data[data['label'] == 1].sample(n=n_sample)

data1 = pd.concat([df_neg, df_pos], ignore_index=True)

In [5]:
data.label.value_counts()

0    5000
1    5000
Name: label, dtype: int64

In [6]:
data1.label.value_counts()

0    500
1    500
Name: label, dtype: int64

In [7]:
data1['polarite'] = data1['label'].map({
    1 : 'positive',
    0 : 'negative'
})

In [8]:
data1.polarite.value_counts()

negative    500
positive    500
Name: polarite, dtype: int64

In [9]:
data1 = data1.drop(columns=['label', 'date_du_poste', 'utilisateur'])

In [10]:
data1.head()

,tweet,polarite
0,@rkh_816 No.. he cancelled .... Thinking bout...,negative
1,andy avalos!! this morning you said 'we would ...,negative
2,@kathysena I am on Destroy Twitter. similar to...,negative
3,Sugarland singing &quot;Love Shack&quot; is li...,negative
4,@keishaaa YEY! :-BD! GOOD LUCK TO US BOTH! :&g...,negative


In [11]:
#*****************************************
# Préparation des librairie et des outils
# pour le nettoyage des tweets
#*****************************************

# Expressions régulières
import re

# Pontuations
import string
ponctuations = list(string.punctuation)
print(ponctuations)

# Tokénisation
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Lemmatisation
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

# Charger les stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
mots_vides = stopwords.words('english')
print('\n')
print(mots_vides)

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'wh

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JK253\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JK253\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JK253\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Nettoyer le contenu des tweets

In [12]:
# fonction pour nettoyage de chaque document
# tweet = corps du tweet = document
# ponctuations : liste des ponctuations
# stopwords : liste des stopwords à retirer
# lem : fonction pour la lemmatisation des termes

def clean_tweet(tweet, ponctuations, stopwords, lem):
    # Harmonisation de la casse
    temp = tweet.lower()
    # retier les contractions en anglais
    temp = re.sub("'", "", temp)
    # retrait des @
    temp = re.sub("@[A-Za-z0-9_]+", "", temp)
    # retrait des #
    temp = re.sub("#[A-Za-z0-9_]+", "", temp)
    # retrait des liens web (http et https)
    temp = re.sub(r'http\S+','', temp)
    # retrait des ponctuations
    temp = "".join([char for char in list(temp) if not (char in ponctuations)])
    # retrait des nombres
    temp = re.sub("[0-9]", "", temp)
    # tokénisation
    temp = word_tokenize(temp)
    # lemmatisation des termes
    temp = [lem.lemmatize(mot) for mot in temp]
    # retrait des stopwords
    temp = [mot for mot in temp if not mot in stopwords]
    # retirer les tokens de moins de 3 caractères
    temp = [mot for mot in temp if len(mot) >= 3]
    return temp

In [13]:
# Appliquer le nettoyage au corpus
corpus = list(data1.tweet)
corpus = [clean_tweet(doc, ponctuations, mots_vides, lem) for doc in corpus]

In [14]:
corpus[0]

['cancelled', 'thinking', 'bout', 'watching', 'movie', 'day', 'mrw']

In [16]:
# Nouvelle dataframe
df_1 = pd.DataFrame({'label': data1.polarite ,'tweet': corpus})
print("avant:.........")
data1.head(10)

avant:.........


,tweet,polarite
0,@rkh_816 No.. he cancelled .... Thinking bout...,negative
1,andy avalos!! this morning you said 'we would ...,negative
2,@kathysena I am on Destroy Twitter. similar to...,negative
3,Sugarland singing &quot;Love Shack&quot; is li...,negative
4,@keishaaa YEY! :-BD! GOOD LUCK TO US BOTH! :&g...,negative
5,"@PinkTrees: i'm going to lay down for a while,...",negative
6,@AbbiKakez No hit up?,negative
7,I just heard that there is a divorce in the fa...,negative
8,I miss my Allen,negative
9,@emilio_delgado I miss you so much,negative


In [17]:
print("après:.........")
df_1.head(10)

après:.........


,label,tweet
0,negative,"[cancelled, thinking, bout, watching, movie, d..."
1,negative,"[andy, avalos, morning, said, would, see, dry,..."
2,negative,"[destroy, twitter, similar, tweetdeckno]"
3,negative,"[sugarland, singing, quotlove, shackquot, like..."
4,negative,"[yey, good, luck, haha, aww, aryt, see, tomorr..."
5,negative,"[going, lay, crampy, feeling, yuck, really, ha..."
6,negative,[hit]
7,negative,"[heard, divorce, family, sort, thing, sooooo, ..."
8,negative,"[miss, allen]"
9,negative,"[miss, much]"


In [18]:
# des tweets vides après nettoyage?
print(df_1.loc[df_1.tweet==""].shape[0])

0


In [19]:
# retrait des tweets correspondants
df_ok = df_1.loc[df_1.tweet != ""]
print(df_ok.shape)

(1000, 2)


## Sauvegarder ma base de données en fichier CSV

In [20]:
data.to_csv (r'P7_03_fichiercsv_tweet_red_simple_pl.csv', index = False, header=True)

In [21]:
data1.to_csv (r'P7_03_fichiercsv_tweet_red_simple_cat_test_1000.csv', index = False, header=True)

In [22]:
df_ok.to_csv (r'P7_03_fichiercsv_tweet_red_simple_net_cat_test_1000.csv', index = False, header=True)